In [4]:
!pip3 install graphviz
!pip3 install anytree
!pip3 install transformers

In [5]:
import pandas as pd
import datetime
import time
from functions import *

import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, InputExample, InputFeatures

from anytree import Node, RenderTree, search
from anytree.exporter import DotExporter
from graphviz import Source

ModuleNotFoundError: No module named 'graphviz'

# Load Subjectivity Detection Model

In [2]:
tokenizer = BertTokenizer.from_pretrained('subjectivity_tokenizer')
model = TFBertForSequenceClassification.from_pretrained('saved_subjectivity_model')

model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at saved_subjectivity_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


# Read the dataset of comments

In [3]:
df = pd.read_csv('Datasets/reddit_data.csv', dtype=object)
df['id'] = df['id'].astype(str)
df['reply_to'] = df['reply_to'].astype(str)
df.head()

,user_name,id,timestamp,reply_to,comment,url,link_title
0,MapleViolet,hpr2kav,2021-12-24 08:55:24,rmqevj,All I know is - anyone trying to pull a fast o...,NaN,NaN
1,HaddockFillet,hra9zzo,2022-01-05 08:14:35,hpr2kav,Why does she think it is OK to lie about such ...,NaN,NaN
2,applescript16,hpntm2t,2021-12-23 16:24:16,rmqevj,Here’s some perspective: \n\n1) The public nat...,NaN,NaN
3,iluj13,hpnwekg,2021-12-23 17:01:54,hpntm2t,Well said. It’s only a problem if your party i...,NaN,NaN
4,forzenrose,hpnzb4r,2021-12-23 17:41:51,hpnwekg,&gt;Transparency and finding out the truth is ...,NaN,NaN


# Construct tree of Comments

In [4]:
parent = 'rmqevj'
root = Node(parent)

input_list = [] 
item_count = df['reply_to'].value_counts().to_dict()

for i in range(len(df['id'].tolist())):
    if df['id'].loc[i] != df['reply_to'].loc[i]:
        input_list.append((df['id'].loc[i], df['reply_to'].loc[i]))

In [5]:
output_dict = make_map(input_list)
createTree(output_dict[parent], root)

In [6]:
printGraph(root)

rmqevj
├── hpr2kav
│   └── hra9zzo
├── hpntm2t
│   ├── hpnwekg
│   │   ├── hpnzb4r
│   │   │   └── hpo5qkg
│   │   │       └── hpokj06
│   │   │           ├── hpp0qgm
│   │   │           ├── hpoqxz5
│   │   │           └── hpraj0s
│   │   │               └── hpshgjh
│   │   ├── hpo2pl5
│   │   │   └── hpo7oxl
│   │   │       └── hpo7u0x
│   │   ├── hpo76sq
│   │   ├── hpo7l9d
│   │   ├── hpradku
│   │   │   ├── hps0rop
│   │   │   └── hprqpon
│   │   ├── hpoya8b
│   │   └── hpnwve6
│   │       └── hpnz3vc
│   │           ├── hpp0vzg
│   │           └── hpo6fwl
│   │               └── hpp7l4j
│   ├── hpo1src
│   │   ├── hpolpfk
│   │   ├── hpo33t8
│   │   │   ├── hpo4que
│   │   │   │   └── hpom3uq
│   │   │   │       └── hprayhb
│   │   │   │           └── hpruuh0
│   │   │   └── hpoq8hi
│   │   │       └── hppkszp
│   │   │           └── hpppv0m
│   │   └── hpo6898
│   │       └── hpshl11
│   ├── hpnx97j
│   │   ├── hpnxtjg
│   │   │   ├── hpo0tg0
│   │   │   │   ├── hpo1tc3
│   │   │

# Predict the results of sentiment analysis

Lexicon-based approaches

In [7]:
pred_sentences = cleanComments(df['comment'])

textblob_polarity = []
textblob_subjectivity = []
vader_results = []
vaderCompoundScores = []

for i in pred_sentences:
    result = getSentimentalResults(sid_obj, i)
    textblob_polarity.append(result[0])
    textblob_subjectivity.append(result[1])
    vader_results.append(result[2])
    vaderCompoundScores.append(result[3])

Supervised learning approach through BERT

In [8]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

labels = [0,1]
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()

result = []

for i in range(len(pred_sentences)):
    result.append(float(tf_predictions[i,1]))

Implement text clustering and save all results into a structured dataframe

In [9]:
df['textblob_polarity'] = textblob_polarity
df['textblob_subjectivity'] = textblob_subjectivity
df['vader_sentiment'] = vader_results
df['vader_compound_score'] = vaderCompoundScores
df['model_subjectivity'] = result
df['topic_cluster'] = getClusters(pred_sentences, embedder)

df.head()

C:\Users\tkjie\anaconda3\envs\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,user_name,id,timestamp,reply_to,comment,url,link_title,textblob_polarity,textblob_subjectivity,vader_sentiment,vader_compound_score,model_subjectivity,topic_cluster
0,MapleViolet,hpr2kav,2021-12-24 08:55:24,rmqevj,All I know is - anyone trying to pull a fast o...,NaN,NaN,4.500000e-01,0.600000,"{'neg': 0.0, 'neu': 0.912, 'pos': 0.088}",0.4404,0.998633,1
1,HaddockFillet,hra9zzo,2022-01-05 08:14:35,hpr2kav,Why does she think it is OK to lie about such ...,NaN,NaN,2.500000e-01,0.500000,"{'neg': 0.0, 'neu': 0.827, 'pos': 0.173}",0.4466,0.000084,0
2,applescript16,hpntm2t,2021-12-23 16:24:16,rmqevj,Here’s some perspective: \n\n1) The public nat...,NaN,NaN,8.214286e-02,0.388095,"{'neg': 0.05, 'neu': 0.828, 'pos': 0.122}",0.8519,0.829385,0
3,iluj13,hpnwekg,2021-12-23 17:01:54,hpntm2t,Well said. It’s only a problem if your party i...,NaN,NaN,2.142857e-01,0.736508,"{'neg': 0.136, 'neu': 0.673, 'pos': 0.191}",0.2263,0.700985,0
4,forzenrose,hpnzb4r,2021-12-23 17:41:51,hpnwekg,&gt;Transparency and finding out the truth is ...,NaN,NaN,3.700743e-17,0.755556,"{'neg': 0.192, 'neu': 0.603, 'pos': 0.205}",-0.1280,0.000424,0
